In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import spacy
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import networkx as nx
from sklearn.neighbors import NearestNeighbors

In [2]:
dtype_dict = {3: 'str'}
df = pd.read_csv('dataset/amazon_data.csv', dtype=dtype_dict, low_memory=False)
df.head(3)

,Unnamed: 0.1,Unnamed: 0,title,original_price,sale_price,rating,review_count,main_category,sub_category_1,sub_category_2,...,is_available,predicted_trendiness,predicted_uniqueness,trendiness_score,uniqueness_score,ecommerce_text,product_embedding,Deliver to Singapore,Deliver to Malaysia,Delivery time
0,0,0,"Muslin Nursing Cover for Breastfeeding, Breath...",$12.99,$11.99,4.6,39,Baby Products,Baby,Privacy Nursing Covers,...,In Stock,0.346864,0.865101,0.159482,0.397760,passage: muslin nursing cover for breastfeedin...,"[0.020546678453683853, 0.04206167161464691, 0....",yes,no,10
1,1,1,Baby Sun Hat UPF 50+ Sun Protective Toddler Bu...,NaN,NaN,4.8,1739,Baby Products,"Clothing, Shoes & Jewelry",Baby Boys' Hats & Caps,...,Not Available,0.442018,0.947105,0.300501,0.719134,passage: baby sun hat upf sun protective todd...,"[0.02578025683760643, 0.045276544988155365, 0....",no,no,14
2,2,2,Dr. Brown's Milestones Wide-Neck Sippy Spout B...,NaN,$8.99,4.7,16812,Baby Products,Baby,Baby Bottles,...,In Stock,0.476247,0.886945,0.465965,0.483366,passage: dr browns milestones wideneck sippy s...,"[0.01611621305346489, 0.05096699669957161, 0.0...",yes,yes,6


In [3]:
import nltk
nltk.download('stopwords')

nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
""""
def get_tfidf(product_details):
    clean_product = []
    product_name = list(product_details)
    for i in range(len(product_name)):
        words = ""

        doc = nlp(product_name[i].lower())
        for token in doc:
            token.lemma_ = re.sub(r'\W',' ',token.lemma_)
            token.lemma_ = token.lemma_.strip()
            if not token.lemma_.endswith("ml") and not token.lemma_.endswith("ms") and not token.lemma_.isdigit() and not token.lemma_ in stop_words:
                if len(token.lemma_) > 2 or token.lemma_ == 'uv': 
                    words += token.lemma_.lower() + " "
                    

        if len(words) > 0:
            clean_product.append(str(words.strip()))

    tfidf_vectorizer=TfidfVectorizer(use_idf=True) 
    tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(clean_product)
    first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]

    df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), 
                      columns=["tfidf"]) 
    df = df.sort_values(by=["tfidf"], ascending=False).reset_index()
    
    return df
    """
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

def get_tfidf(documents):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(documents)
    first_vector_tfidfvectorizer = tfidf_vectorizer_vectors[0]
    
    df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), 
                      index=tfidf_vectorizer.get_feature_names_out(), 
                      columns=["tfidf"]) 
    df = df.sort_values(by=["tfidf"], ascending=False).reset_index()
    
    return df

In [20]:
G = nx.Graph()
# entries = os.listdir('dataset/updated/') ## Please Change Directory
main_category_list = []
sub_category_list = []
sub_category_list_2 = []
product_name_list = []

df['main_category'] = df['main_category'].str.lower()
df['sub_category_1'] = df['sub_category_1'].str.lower()
df['sub_category_2'] = df['sub_category_2'].str.lower()

main_category = df['main_category'].unique()
for _main_category in main_category:
    if type(_main_category) == str:
        print(_main_category)
        main_category_data = df[df['main_category'] == _main_category]
        G.add_node(_main_category)
        main_category_list.append(_main_category)
        
        for row in main_category_data['sub_category_1'].unique():
            if type(row) == str:
                sub_category_list.append(row)
                G.add_edge(row, _main_category, weight=1.0)
                
                for row2 in main_category_data.loc[main_category_data['sub_category_1'] == row]['sub_category_2'].unique():
                    if type(row2) == str:
                        G.add_edge(row2.strip(), row.strip(), weight=1.0)
                        sub_category_list_2.append(row2.strip())
                        
                        tfidf_result = get_tfidf(main_category_data.loc[main_category_data['sub_category_2'] == row2]['title'])
                        
                        index =  tfidf_result['index']
                        tfidf = tfidf_result['tfidf']
                        counter = 0
                        for _tfidf_result in tfidf_result['index']:
                            if float(tfidf[counter]) > 0.0:                    
                                if index[counter] not in main_category_list:
                                    product_name_list.append(index[counter].lower())
                                    G.add_edge(index[counter].lower(), row2.strip(), weight=tfidf[counter])
    
    
                            counter +=1

sports and outdoors
electronics
home and kitchen
computers
beauty and personal care
baby products
boy's fashion
girl's fashion
mens's fashion
main_category
toys and games
women's fashion


In [5]:
import pickle

#generate
#with open('network_theory.pickle','wb') as fe_data_file:
#     pickle.dump(G, fe_data_file)

#load
with open('dataset/network_theory.pickle', 'rb') as fe_data_file:
    G = pickle.load(fe_data_file)


In [17]:
import json
BETWEENNESS_CENTRALITY = nx.betweenness_centrality(G)
with open('dataset/betweenness_centrality.json', 'w') as json_file:
    json.dump(BETWEENNESS_CENTRALITY, json_file)

In [19]:
with open('dataset/betweenness_centrality.json', 'r') as json_file:
    BETWEENNESS_CENTRALITY = json.load(json_file)

In [7]:
# BETWEENNESS_CENTRALITY = nx.betweenness_centrality(G)

#with open('betweeness_centrality.pickle','wb') as fe_data_file:
#     pickle.dump(G, fe_data_file)


# Load the betweenness centrality from the pickle file
with open('dataset/betweeness_centrality.pickle', 'rb') as fe_data_file:
    BETWEENNESS_CENTRALITY = pickle.load(fe_data_file)

In [20]:
def recommend_products(given_wishlist, G, BETWEENNESS_CENTRALITY, overall_data):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(given_wishlist.strip())
    sub_category_list_2 = list(overall_data['sub_category_2'].str.lower().unique())
    result_categories = []
    
    for token in reversed(doc):
        if token.text in list(G.nodes()):
            closeness_centrality_list = []
            betweness_centrality_list = []
            neighbor_list = []
            shortest_path_list = []
            length_list = []
            
            for neighbor in list(G.neighbors(token.text)):
                if neighbor in sub_category_list_2:
                    neighbor_list.append(neighbor)
                    betweness_centrality_list.append(BETWEENNESS_CENTRALITY[neighbor])
                    shortest_path = nx.shortest_path(G, source=neighbor, target=token.lemma_)
                    shortest_path_list.append(len(shortest_path))
                    length_list.append(overall_data[overall_data['sub_category_2'] == neighbor].shape[0])
                    
            network_result = pd.DataFrame(neighbor_list, columns=['neighbor'])
            network_result['betweenness_centrality'] = betweness_centrality_list
            network_result['shortest_path'] = shortest_path_list
            
            if betweness_centrality_list:
                if network_result[network_result['shortest_path'] == min(shortest_path_list)]['neighbor'].shape[0] < 2:
                    if list(network_result[network_result['shortest_path'] == min(shortest_path_list)]['neighbor'])[0] not in result_categories:
                        result_categories.append(list(network_result[network_result['shortest_path'] == min(shortest_path_list)]['neighbor'])[0])
                else:
                    if list(network_result[network_result['betweenness_centrality'] == min(betweness_centrality_list)]['neighbor'])[0] not in result_categories:
                        result_categories.append(list(network_result[network_result['betweenness_centrality'] == min(betweness_centrality_list)]['neighbor'])[0])
    
    merge_products = []
    for result_category in result_categories:
        merge_products.append(overall_data[overall_data['sub_category_2'] == result_category.title()])
    
    selected_category = pd.concat(merge_products).reset_index()
    
    return selected_category

In [22]:
#Example usage
given_wishlist = 'sports'
overall_data = pd.read_csv('dataset/product_sample.csv')  # Load your overall data
selected_category = recommend_products(given_wishlist, G, BETWEENNESS_CENTRALITY, overall_data)
selected_category

,index,Unnamed: 0.1,Unnamed: 0,title,original_price,sale_price,rating,review_count,main_category,sub_category_1,...,is_available,predicted_trendiness,predicted_uniqueness,trendiness_score,uniqueness_score,ecommerce_text,product_embedding,Deliver to Singapore,Deliver to Malaysia,Delivery time
0,1535,1535,1535,PUMA Boys Pure Core Soccer Pant,$22.69,$20.34,4.5,1184,Boy's Fashion,"Clothing, Shoes & Jewelry",...,In Stock,0.309869,0.820108,0.338315,0.221434,passage: puma boys pure core soccer pant,"[0.02635507844388485, 0.03706911578774452, 0.0...",no,no,11
1,1932,1932,1932,adidas Boys' Iconic Tricot Jogger Pants with D...,$35.00,$20.87,4.4,13,Boy's Fashion,"Clothing, Shoes & Jewelry",...,In Stock,0.262523,0.768136,0.567188,0.017760,passage: adidas boys iconic tricot jogger pant...,"[0.022459017112851143, 0.036920737475156784, 0...",yes,no,8
2,1968,1968,1968,adidas Boys' Iconic Tricot Jogger Pants,NaN,$35.00,4.6,2996,Boy's Fashion,"Clothing, Shoes & Jewelry",...,In Stock,0.359308,0.862090,0.099324,0.385961,passage: adidas boys iconic tricot jogger pants,"[0.023631906136870384, 0.03746218606829643, 0....",yes,yes,5


In [26]:
"""
def calculate_relevance(sample_wishlist, selected_category):
    vectorize = TfidfVectorizer(stop_words='english')
    tfidf_response = vectorize.fit_transform(selected_category['title'])
    dtm = pd.DataFrame(tfidf_response.todense(), columns=vectorize.get_feature_names())
    
    nn = NearestNeighbors(n_neighbors=selected_category.shape[0])
    nn.fit(dtm)
    
    new = vectorize.transform(sample_wishlist)
    knn_model_result = nn.kneighbors(new.todense())
    
    knn_result = pd.DataFrame(knn_model_result[0][0], columns=['Distance'])
    knn_result["title"] = selected_category['title'].iloc[knn_model_result[1][0]]
    
    merged_result = pd.merge(selected_category, knn_result, on='title', how='inner')
    merged_result = merged_result.drop_duplicates(subset='title', keep="first")
    
    return merged_result
"""
#Example usage function
def calculate_relevance(wishlist, category_data):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_response = vectorizer.fit_transform(category_data['title'])
    dtm = pd.DataFrame(tfidf_response.todense(), columns=vectorizer.get_feature_names_out())
    
    nn = NearestNeighbors(n_neighbors=category_data.shape[0])
    nn.fit(dtm)
    
    wishlist_tfidf = vectorizer.transform(wishlist)
    distances, indices = nn.kneighbors(wishlist_tfidf)
    
    final_recommendations = category_data.iloc[indices[0]]
    
    return final_recommendations

In [33]:
#Example usage
sample_wishlist = ['glow']
final_recommendations = calculate_relevance(sample_wishlist, selected_category)
final_recommendations

C:\Users\Jon Ang\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,index,title,original_price,sale_price,discount,rating,review_count,main_category,sub_category_1,sub_category_2,rankings,description,insights,purchase_cnt_prev_month,store_name,supplier,is_available,url
2,21041,111 Glow in The Dark Stars for Ceiling with Bo...,$9.99,$5.99,NaN,4.1,176,Baby Products,Toys & Games,Baby & Toddler Toys,"#4,995 in Toys & Games (See Top 100 in Toys & ...",['😎 GLOW IN THE DARK STARS FOR CEILING with Bo...,"['Quality_POSITIVE', 'Brightness_NEGATIVE']",500+ bought in past month,Happy Hippo,Hoboken Gifts & Supplies,In Stock,/Happy-Hippo-111-Pack-Christmas-Stocking/dp/B0...
1,21000,111 Glow in The Dark Stars for Ceiling with Bo...,$9.99,$5.99,NaN,4.1,176,Baby Products,Toys & Games,Baby & Toddler Toys,"#4,995 in Toys & Games (See Top 100 in Toys & ...",['😎 GLOW IN THE DARK STARS FOR CEILING with Bo...,"['Quality_POSITIVE', 'Brightness_NEGATIVE']",500+ bought in past month,Happy Hippo,Hoboken Gifts & Supplies,In Stock,/Happy-Hippo-111-Pack-Christmas-Stocking/dp/B0...
3,21201,Melissa & Doug Children's Book - Poke-a-Dot: A...,$13.99,$12.21,NaN,4.7,"4,093",Baby Products,Toys & Games,Baby & Toddler Toys,"#6,336 in Toys & Games (See Top 100 in Toys & ...","['90% cardboard', 'Imported', '20-page interac...","['Interactive elements_POSITIVE', 'Quality_MIX...",1K+ bought in past month,Melissa & Doug Store,NaN,In Stock,/Melissa-Doug-Childrens-Alphabet-Toddlers/dp/1...
7,23044,Melissa & Doug Toy Keys and Cars Wooden Rescue...,NaN,NaN,NaN,4.7,"2,718",Baby Products,Toys & Games,Baby & Toddler Toys,"#8,631 in Toys & Games (See Top 100 in Toys & ...","['Three-bay garage with locking doors, color-c...","['Quality_POSITIVE', 'Motor skills_POSITIVE', ...",NaN,Melissa & Doug Store,NaN,Not Available,/Melissa-Doug-Wooden-Rescue-Vehicle/dp/B071DTR...
9,35303,"Toys for 1-10 Year Old Girls,Star Projector fo...",$17.99,$9.99,44.0,4.2,"4,211",Toys and Games,Toys & Games,Baby & Toddler Toys,#225 in Toys & Games (See Top 100 in Toys & Ga...,"['UNIQUE DESIGN -- Unlike other products, our ...","['Appearance_POSITIVE', 'Color settings_POSITI...",4K+ bought in past month,HONGID Store,ToyKidsDirect,In Stock,/Projector-Christmas-Birthday-Stocking-Stuffer...
5,21414,"Sesame Street Elmo's On The Go Letters, 24 x 3...",NaN,$23.50,NaN,4.8,"10,956",Baby Products,Toys & Games,Baby & Toddler Toys,"#6,721 in Toys & Games (See Top 100 in Toys & ...",['Learn the alphabet with 26 colorful letters'...,"['Quality_POSITIVE', 'Sturdiness_POSITIVE', 'A...",900+ bought in past month,Sesame Street Store,Bo Toys and Gifts,In Stock,/Sesame-Street-Elmos-Go-Letters/dp/B00PAE805S/...
0,19439,"Toys for 1-10 Year Old Girls,Star Projector fo...",$17.99,$12.99,NaN,4.2,"4,178",Baby Products,Toys & Games,Baby & Toddler Toys,#709 in Toys & Games (See Top 100 in Toys & Ga...,"['UNIQUE DESIGN -- Unlike other products, our ...","['Appearance_POSITIVE', 'Color settings_POSITI...",3K+ bought in past month,HONGID Store,ToyKidsDirect,In Stock,/Projector-Christmas-Birthday-Stocking-Stuffer...
10,35643,Sassy Teethe & Twirl Sensation Station 2-in-1 ...,$13.99,$9.97,29.0,4.6,"3,382",Toys and Games,Toys & Games,Baby & Toddler Toys,"#1,063 in Toys & Games (See Top 100 in Toys & ...","['Soft, textured surfaces encourage mouthing, ...","['Educational_POSITIVE', 'Versatility_POSITIVE...",3K+ bought in past month,Sassy Store,NaN,In Stock,/Sassy-80680-Teethe-Twirl-Station/dp/B01LZEEQSW
6,21446,"Sesame Street Elmo's On The Go Letters, 24 x 3...",NaN,$23.50,NaN,4.8,"10,956",Baby Products,Toys & Games,Baby & Toddler Toys,"#6,721 in Toys & Games (See Top 100 in Toys & ...",['Learn the alphabet with 26 colorful letters'...,"['Quality_POSITIVE', 'Sturdiness_POSITIVE', 'A...",900+ bought in past month,Sesame Street Store,Bo Toys and Gifts,In Stock,/Sesame-Street-Elmos-Go-Letters/dp/B00PAE805S/...
4,21323,Melissa & Doug K’s Kids My First Activity Book...,NaN,$16.99,NaN,4.7,"3,420",Baby Products,Toys & Games,Baby & Toddler Toys,"#7,370 in Toys & Games (See Top 100 in Toys & ...",['K’s kids cloth 